In [1]:
# Problem 1: Stacking packages

# Given a set of n packages with length, breadth, and weight L[i], B[i], and W[i], what is the tallest
# possible stack of packages.

# A box can be placed on top of another if it is strictly smaller (it can be rotated though)
# and strictly lighter in weight

# returns if the second box can be stacked on the first
def IsStackable(L, B, W, i1, i2):
    return (W[i2] < W[i1]) and ((L[i2] < L[i1] and B[i2] < B[i1]) or (B[i2] < L[i1] and L[i2] < B[i1]))

# Recurrence Equation:
# let p(i) be all packages with index k such that IsStackable(i, k) is true, then 
# OPT(i) = max{OPT(k)} + 1    if there are no k such that P(k) is true, max{OPT(k)} returns 0
#          k in p(i)
# OPT(0) = 0, OPT(1) = 1

# returns the indices of the boxes that are part of the tallest stack
def TallestStack(L, B, W, n):
    # sort L, B, W in order of increasing weight
    OPT = [0] * n # dp[i] = max stack starting with box i
    box_above = [-1] * (n) # box_above[i] = the box placed above i in the max stack of i
    tallest_stack = 0;
    for i in range(0, n): # for i from 1 to n
        for k in range(0, i):  # for k from 1 to i-1
            if IsStackable(L, B, W, i, k) and (dp[k] > dp[box_above[i]]):
                box_above[i] = k
        dp[i] = 1 + dp[box_above[i]]
        if dp[i] > dp[tallest_stack]:
            tallest_stack = i
    
    optimal = []
    while tallest_stack >= 0:
        optimal.append(tallest_stack)
        tallest_stack = box_above[tallest_stack]
    
    return optimal

L = [5, 3, 8, 7, 13]
B = [2, 1, 6, 9, 14]
W = [1, 2, 3, 4, 6]

TallestStack(L, B, W, 5)

[4, 3, 2, 0]

In [2]:
# Problem 2: Nicest photos

# The skyline has n buildings of height h[n]
# Someone can take photos with k of the buildings in them, but without capturing the same building twice
# The score for a photo is the sum of heights + the max height

# calculate the score of all possible photos from building i-k+1 to i
def Scores(h, k):
    scores = [0] * len(h)
    for i in range(k-1, len(h)): # for i from k to n
        heights = []
        for j in range (i, i-k, -1): # for j from i down to i-k+1
            heights.append(h[j])
        scores[i] = max(heights) + sum(heights)
    
    return scores


# Recurrence Equation:
# OPT(i) = max{score[i] + OPT(i - k), OPT(i - 1)}
# OPT(i <= 0) = 0


# find the best possible score of non-overlapping photos with k buildings
# and return the location of the photos
def NicestPhotos(h, k):
    scores = Scores(h, k)
    OPT = [0] * len(h) # d
    for i in range(len(h)): # for i from 1 to n
        if i-k+1 < 0:  # if i-k < 0
            OPT[i] = 0
        else:
            OPT[i] = max(scores[i] + OPT[i-k], OPT[i-1])
            
    photos = []
    j = len(h) - 1 # j = n
    while j >= 0: # while j > 0
        if OPT[j] > OPT[j-1]: # jth picture was included
            photos.append(j)
            j = j -k
        else:
            j = j - 1
    
    return OPT[len(h) -1], photos # OPT[n] + photos

h = [3, 7, 4, 2, 8, 9, 4, 1, 6, 5]
NicestPhotos(h, 3)

(69, [9, 6, 2])

In [3]:
# Problem 3: Elegant subsequences

# We are given a sorted array, an elegant subsequence is one where the difference
# between consecutive elements is constant

# Recurrence Equation:
# L[i,j] is the length of the longest elegant subsequence whose last 2 indices are i and j
# p[i,j] is the index of an element in A such that A[p[i,j]] = 2A[i] - A[j]

# L[i,j] = 1 + L[p[i,j], j]     if p[i,j] exists
#        = 2                    if p[i,j] does not exist


def ElegantSubsq(A):
    n = len(A)
    L = [[0 for i in range(n)] for j in range(n)] # n x n array where L[i, j] is row i and column j
    max_i = 0
    max_i = 1
    max_length = -1
    for i in range(n - 1): # for i from 1 to n-1
        for j in range(i+1, n): # for j from i+1 to n
            target = 2*A[i] - A[j]
            
            # this can be replaced with a binary search that returns
            # the index of the element in A that equals target
            p = -1
            for k in range(i): # for p from 1 to i-1
                if A[k] == target:
                    p = k
            if p == -1:
                L[i][j] = 2
            else:
                L[i][j] = 1 + L[p][i]
            if L[i][j] > max_length:
                max_i = i
                max_j = j
                max_length = L[i][j]
                
    diff = A[max_j] - A[max_i]
    last = A[max_i]
    sequence = []
    sequence.insert(0, A[max_j])
    sequence.insert(0, A[max_i])
    for i in range(max_i-1, 0-1, -1):
        if last - A[i] == diff:
            sequence.insert(0, A[i])
            last = A[i]
    return sequence

A = [1, 3, 4, 7, 8, 10, 12, 17]
ElegantSubsq(A)

[1, 4, 7, 10]

In [35]:
# Problem 4:

# You have a video consisting of n frame and each one takes s[0...n-1] bandwith.
# You want to partition the video into at most k segments of any size such that the total
# amount of bandwith needed is minimized. The bandwith required for any segment is the number
# of frames in the segment times the frame in the segment that requires the most bandwith.

# Recurrence Equation:
# let cost(a,b) be the cost of the segment from frame a to frame b. i = -1 indicates no frames
# OPT(i, m) = min{OPT(j, m-1) + cost(j+1, i)}    if there are no k such that P(k) is true, max{OPT(k)} returns 0
#          -1<=j<i
# OPT(0) = 0, OPT(1) = 1

def Cost(S, a, b):
    return len(S[a:b+1]) * max(S[a:b+1])

def MinCost(S, k):
    OPT = [[0 for i in range(k)] for j in range(len(S))]
    FrameBefore = [[0 for i in range(k)] for j in range(len(S))]
    for i in range(len(S)): # for i from 0 to n-1
        for m in range(0, k): # for m from 1 to k
            minn = float('inf')
            for j in range(-1, i): # for j from -1 to i-1
                cost = 0
                if j == -1 or m == 0: # if j == -1 or m == 1
                    cost = Cost(S, 0, i)
                else:
                    cost = OPT[j][m-1] + Cost(S, j+1, i)
                if cost < minn:
                    minn = cost
                    FrameBefore[i][m] = j
            OPT[i][m] = minn
    
    p = []
    frame = len(S) - 1 # frame = n - 1
    for seg in range(k-1, 0, -1): # for seg from k down to 2
        p.insert(0, FrameBefore[frame][seg] + 1)
        frame = FrameBefore[frame][seg]
    
    return OPT[len(S)-1][k-1], p       
        
                                   # indices  I_0: [0-0]     I_1: [1 - 5]   I_3: [6 - 7]    I_3: [8 - 9]
S = [3, 8, 2, 4, 6, 9, 2, 3, 8, 5] # values      [3]      [8, 2, 4, 5, 9]      [2, 3]         [8 5]
MinCost(S, 4)                      # costs        3        9*5 = 45             3*2 = 6        8*2 = 16   Total: 70

(70, [1, 6, 8])

In [64]:
# Midterm review question: Capturing treasure 

# With a bag of weight W, and items of weight w[i], how can we fill the exact
# weight of the bag with the fewest items as possible. Items can repeat.

def Treasure(w: list, W: int) -> list:
    dp = [0] * (W+1)
    for i in range(1, W+1):
        options = [float('inf')]
        for k in w:
            if k <= i:
                options.append(dp[i-k])
        dp[i] = 1 + min(options)
    return Reconstruct(w, W, dp)

def Reconstruct(w, W, dp):
    final = []
    while W > 0:
        for j in w:
            if j <= W and (dp[W] == (1 + dp[W-j])):
                final.append(j)
                W -= j
                break
    return final

Treasure([1, 4, 5], 18)

[4, 4, 5, 5]